In [1]:
import random
import time
import pandas as pd
from deap import creator, base, tools, algorithms

# Lista de itens: ID, Nome do item, Valor (R$), Peso (kg)
items = [
    (1, "Barraca", 150.0, 3.5),
    (2, "Saco de dormir", 100.0, 2.0),
    (3, "Isolante térmico", 50.0, 0.5),
    (4, "Colchão inflável", 80.0, 1.0),
    (5, "Lanterna", 30.0, 0.2),
    (6, "Kit de primeiros socorros", 20.0, 0.5),
    (7, "Repelente de insetos", 15.0, 0.1),
    (8, "Protetor solar", 20.0, 0.2),
    (9, "Canivete", 10.0, 0.1),
    (10, "Mapa e bússola", 25.0, 0.3),
    (11, "Garrafa de água", 15.0, 1.8),
    (12, "Filtro de água", 50.0, 0.5),
    (13, "Comida (ração liofilizada)", 50.0, 3.0),
    (14, "Fogão de camping", 70.0, 1.5),
    (15, "Botijão de gás", 30.0, 1.2),
    (16, "Prato, talheres e caneca", 20.0, 0.5),
    (17, "Roupas (conjunto)", 80.0, 1.5),
    (18, "Calçados (botas)", 120.0, 2.0),
    (19, "Toalha", 20.0, 0.5),
    (20, "Kit de higiene pessoal", 30.0, 0.5),
]

# Capacidade máxima da mochila em kg
max_weight = 15.0

# Definir os parâmetros para o grid search
param_grid = {
    "population_size": [50, 100, 200],
    "num_generations": [50, 100, 200],
    "crossover_probability": [0.5, 0.7, 0.9],
    "mutation_probability": [0.1, 0.2, 0.3],
    "tournament_size": [2, 5, 10],
    "select_method": ["selTournament", "selRoulette"],
}


# Função para rodar o algoritmo genético com uma combinação de parâmetros
def run_genetic_algorithm(params):

    population_size = params["population_size"]
    num_generations = params["num_generations"]
    crossover_probability = params["crossover_probability"]
    mutation_probability = params["mutation_probability"]
    tournament_size = params["tournament_size"]
    select_method = params["select_method"]

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register(
        "individual",
        tools.initRepeat,
        creator.Individual,
        toolbox.attr_bool,
        n=len(items),
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def eval_knapsack(individual):
        # Cálculo do valor e peso total dos itens selecionados
        value = sum(items[i][2] for i in range(len(individual)) if individual[i] == 1)
        weight = sum(items[i][3] for i in range(len(individual)) if individual[i] == 1)
        if weight > 15:  # Respeitar o limite de peso
            return (0,)
        return (value,)

    toolbox.register("evaluate", eval_knapsack)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=mutation_probability)

    # toolbox.register("select", getattr(tools, select_method), tournsize=tournsize)

    # Escolher a função de seleção com base no método escolhido
    if select_method == "selTournament":
        toolbox.register("select", tools.selTournament, tournsize=tournament_size)
    elif select_method == "selRoulette":
        toolbox.register("select", tools.selRoulette)
    else:
        raise ValueError("Método de seleção desconhecido!")

    population = toolbox.population(n=population_size)

    for gen in range(num_generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))

    top_individual = tools.selBest(population, 1)[0]
    return top_individual.fitness.values[0]


# Rodar o grid search manualmente
best_score = -1
best_params = {}

# Registra o tempo de início
start_time = time.time()

for population_size in param_grid["population_size"]:
    for NGEN in param_grid["num_generations"]:
        for CXPB in param_grid["crossover_probability"]:
            for MUTPB in param_grid["mutation_probability"]:
                for tournsize in param_grid["tournament_size"]:
                    for select_method in param_grid["select_method"]:
                        params = {
                            "population_size": population_size,
                            "num_generations": NGEN,
                            "crossover_probability": CXPB,
                            "mutation_probability": MUTPB,
                            "tournament_size": tournsize,
                            "select_method": select_method,
                        }
                        score = run_genetic_algorithm(params)
                        print(f"Parâmetros: {params} => Melhor valor: {score}")

                        if score > best_score:
                            best_score = score
                            best_params = params

# Registra o tempo de término
end_time = time.time()

# Calcula o tempo total de execução
execution_time = end_time - start_time

Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 2, 'select_method': 'selTournament'} => Melhor valor: 845.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 2, 'select_method': 'selRoulette'} => Melhor valor: 780.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 5, 'select_method': 'selTournament'} => Melhor valor: 830.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 5, 'select_method': 'selRoulette'} => Melhor valor: 775.0


c:\ProgramData\Miniconda3\envs\scipy\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\ProgramData\Miniconda3\envs\scipy\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 10, 'select_method': 'selTournament'} => Melhor valor: 840.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.1, 'tournament_size': 10, 'select_method': 'selRoulette'} => Melhor valor: 820.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.2, 'tournament_size': 2, 'select_method': 'selTournament'} => Melhor valor: 860.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.2, 'tournament_size': 2, 'select_method': 'selRoulette'} => Melhor valor: 755.0
Parâmetros: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.2, 'tournament_size': 5, 'select_method': 'selTournament'} => Melhor valor: 780.0
Parâmetros: {'pop

In [2]:
# Imprime os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:", best_params)

Melhores parâmetros encontrados: {'population_size': 50, 'num_generations': 50, 'crossover_probability': 0.5, 'mutation_probability': 0.2, 'tournament_size': 10, 'select_method': 'selTournament'}


In [3]:
# Imprime o tempo de execução
print(f"Tempo de execução (evolução somente): {execution_time:.4f} segundos")

Tempo de execução (evolução somente): 207.2722 segundos
